In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry


In [2]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [3]:
forcast_france=pd.DataFrame({})

In [4]:
	# Make sure all required weather variables are listed here
	# The order of variables in hourly or daily is important to assign them correctly below
for i in range(50,55,1):
		for j in range(-1,5,1):
			url = "https://api.open-meteo.com/v1/forecast"
			params = {
				"latitude": i,
				"longitude": j,
				"hourly": "temperature_2m",
				"models": "meteofrance_arome_france"
			}
			responses = openmeteo.weather_api(url, params=params)
			# Process first location. Add a for-loop for multiple locations or weather models
			response = responses[0]
			# Process hourly data. The order of variables needs to be the same as requested.
			hourly = response.Hourly()
			hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

			hourly_data = {"date": pd.date_range(
				start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
				end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
				freq = pd.Timedelta(seconds = hourly.Interval()),
				inclusive = "left"
			)}
			hourly_data["temperature_2m"] = hourly_temperature_2m
			
			hourly_dataframe = pd.DataFrame(data = hourly_data)
			

			nom='hourly_temperature_2m'+str(i)+str(j)+'.csv'

			hourly_dataframe.to_csv(nom)

			



In [5]:

data_frame_test = pd.read_csv('hourly_temperature_2m513.csv')
print(data_frame_test.head(20))

    Unnamed: 0                       date  temperature_2m
0            0  2024-12-07 00:00:00+00:00        7.513000
1            1  2024-12-07 01:00:00+00:00        8.063000
2            2  2024-12-07 02:00:00+00:00        8.513000
3            3  2024-12-07 03:00:00+00:00        8.613000
4            4  2024-12-07 04:00:00+00:00        8.763000
5            5  2024-12-07 05:00:00+00:00       10.013000
6            6  2024-12-07 06:00:00+00:00       11.063000
7            7  2024-12-07 07:00:00+00:00       10.313000
8            8  2024-12-07 08:00:00+00:00        9.863000
9            9  2024-12-07 09:00:00+00:00        9.013000
10          10  2024-12-07 10:00:00+00:00        8.613000
11          11  2024-12-07 11:00:00+00:00        9.162999
12          12  2024-12-07 12:00:00+00:00        8.662999
13          13  2024-12-07 13:00:00+00:00        8.013000
14          14  2024-12-07 14:00:00+00:00        7.713000
15          15  2024-12-07 15:00:00+00:00        7.713000
16          16

In [ ]:
print(hourly_dataframe.head(50))